In [7]:
# -------------------------------------------------------------
# 작성자 : 백강민
# 작성목적 : SKALA Python Day1 - 대용량 데이터 파이프라인 메모리 프로파일링
# 작성일 : 2025-01-12
# 변경사항 내역 :
#   2025-01-12 - 최초 작성
# -------------------------------------------------------------

import time
import tracemalloc
from typing import Callable, Dict, Tuple, Any


def format_memory(size_bytes: int) -> str:
    kb = size_bytes / 1024
    mb = kb / 1024
    return f"{size_bytes:,} bytes ({kb:.2f} KB, {mb:.2f} MB)"


def format_time(seconds: float) -> str:
    return f"{seconds:.6f} sec ({seconds * 1000:.3f} ms)"


def speed_summary(list_time: float, gen_time: float) -> Tuple[str, str]:
    """측정 결과 기반 속도 요약 (빠름/느림 + 배수)"""
    if list_time <= 0 or gen_time <= 0:
        return "측정값 0 sec", "측정값 0 sec"

    if list_time < gen_time:
        ratio = gen_time / list_time
        return f"더 빠름 (약 {ratio:.2f}x)", f"더 느림 (약 {ratio:.2f}x)"
    if gen_time < list_time:
        ratio = list_time / gen_time
        return f"더 느림 (약 {ratio:.2f}x)", f"더 빠름 (약 {ratio:.2f}x)"
    return "동일", "동일"


def profile_sum_with_tracemalloc(label: str, build_iterable_fn: Callable[[], Any]) -> Dict[str, Any]:
    """
    tracemalloc으로 메모리(현재/피크) + 처리시간을 측정하며 sum 계산
    - build_iterable_fn: iterable을 만들어 반환하는 함수
    """
    tracemalloc.start()
    t_start = time.perf_counter()

    iterable = build_iterable_fn()
    total = sum(iterable)

    t_end = time.perf_counter()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    return {
        "label": label,
        "sum": total,
        "time": t_end - t_start,
        "current_mem": current,
        "peak_mem": peak,
    }


def print_summary_table(result_list: Dict[str, Any], result_gen: Dict[str, Any]) -> None:
    """마지막 요약 표를 '리스트 vs 제너레이터'가 명확히 보이도록 출력"""
    list_label = f"{result_list['label']} [Eager]"
    gen_label = f"{result_gen['label']} [Lazy]"

    list_speed_desc, gen_speed_desc = speed_summary(result_list["time"], result_gen["time"])

    # 표 폭 설정
    left_w = 18
    col_w = 58

    def row(metric: str, left: str, right: str) -> None:
        print(f"{metric:<{left_w}} | {left:<{col_w}} | {right:<{col_w}}")

    print("【 실습 정리 포인트 (측정 결과 기반) 】")
    row("구분", list_label, gen_label)
    print("-" * (left_w + 3 + col_w + 3 + col_w))

    row(
        "메모리 사용량",
        f"Peak {format_memory(result_list['peak_mem'])}",
        f"Peak {format_memory(result_gen['peak_mem'])}",
    )
    row(
        "처리 방식",
        "모든 요소를 한 번에 메모리에 적재 (즉시 평가)",
        "한 개씩 순차 처리 (지연 평가)",
    )
    row(
        "처리 시간",
        f"{format_time(result_list['time'])} / {list_speed_desc}",
        f"{format_time(result_gen['time'])} / {gen_speed_desc}",
    )


# =============================================================
# 실습 조건: 1,000만 개 정수 데이터
# =============================================================
N = 10_000_000  # 0 ~ 9,999,999

print("【 대용량 데이터 파이프라인 메모리 프로파일링 】\n")
print("조건: 0 ~ 9,999,999 (총 1,000만 개) 정수에 대해 '제곱 합(sum of squares)' 계산\n")

# =============================================================
# 1) List Comprehension 방식 (Eager)
# =============================================================
print("1) List Comprehension 방식 측정 (리스트 생성 → sum)\n")

result_list = profile_sum_with_tracemalloc(
    "List Comprehension",
    lambda: [i * i for i in range(N)],
)

print(f"   - 총합(리스트): {result_list['sum']}")
print(f"   - 처리 시간(리스트): {format_time(result_list['time'])}")
print(f"   - 메모리(Current): {format_memory(result_list['current_mem'])}")
print(f"   - 메모리(Peak): {format_memory(result_list['peak_mem'])}\n")

# =============================================================
# 2) Generator Expression 방식 (Lazy)
# =============================================================
print("2) Generator Expression 방식 측정 (지연 생성 → sum)\n")

result_gen = profile_sum_with_tracemalloc(
    "Generator Expression",
    lambda: (i * i for i in range(N)),
)

print(f"   - 총합(제너레이터): {result_gen['sum']}")
print(f"   - 처리 시간(제너레이터): {format_time(result_gen['time'])}")
print(f"   - 메모리(Current): {format_memory(result_gen['current_mem'])}")
print(f"   - 메모리(Peak): {format_memory(result_gen['peak_mem'])}\n")

# =============================================================
# 3) 결과 비교 + Lazy Evaluation 설명
# =============================================================
print("3) 결과 비교 및 Lazy Evaluation(지연 평가) 설명\n")

if result_list["sum"] != result_gen["sum"]:
    print("   [경고] 총합이 다릅니다. 로직을 다시 확인하세요.\n")

time_diff = result_list["time"] - result_gen["time"]
peak_mem_diff = result_list["peak_mem"] - result_gen["peak_mem"]

list_speed_desc, gen_speed_desc = speed_summary(result_list["time"], result_gen["time"])

print(f"   - 처리 시간(리스트): {format_time(result_list['time'])} / {list_speed_desc}")
print(f"   - 처리 시간(제너레이터): {format_time(result_gen['time'])} / {gen_speed_desc}")
print(f"   - 처리 시간 차이(리스트 - 제너레이터): {format_time(time_diff)}\n")

print(f"   - Peak 메모리(리스트): {format_memory(result_list['peak_mem'])}")
print(f"   - Peak 메모리(제너레이터): {format_memory(result_gen['peak_mem'])}")
print(f"   - Peak 메모리 차이(리스트 - 제너레이터): {format_memory(peak_mem_diff)}\n")

print("【 Lazy Evaluation(지연 평가) 요약 】")
print("- List Comprehension: [ ... ] 형태로 '전체 결과를 한 번에' 리스트로 만들고 메모리에 적재한 뒤 처리 (Eager)")
print("- Generator Expression: ( ... ) 형태로 '필요한 순간에 하나씩' 값을 생성하여 처리 (Lazy)")
print("- 따라서 대용량 데이터에서는 제너레이터가 Peak 메모리를 크게 줄이는 경향이 있음")
print("  (대신, 상황에 따라 리스트가 더 빠를 수도/제너레이터가 비슷하거나 느릴 수도 있음)\n")

# =============================================================
# 4) 표로 정리 (라벨 명확히)
# =============================================================
print_summary_table(result_list, result_gen)


【 대용량 데이터 파이프라인 메모리 프로파일링 】

조건: 0 ~ 9,999,999 (총 1,000만 개) 정수에 대해 '제곱 합(sum of squares)' 계산

1) List Comprehension 방식 측정 (리스트 생성 → sum)

   - 총합(리스트): 333333283333335000000
   - 처리 시간(리스트): 5.508550 sec (5508.550 ms)
   - 메모리(Current): 409,110,296 bytes (399521.77 KB, 390.16 MB)
   - 메모리(Peak): 409,110,488 bytes (399521.96 KB, 390.16 MB)

2) Generator Expression 방식 측정 (지연 생성 → sum)

   - 총합(제너레이터): 333333283333335000000
   - 처리 시간(제너레이터): 5.779389 sec (5779.389 ms)
   - 메모리(Current): 1,822 bytes (1.78 KB, 0.00 MB)
   - 메모리(Peak): 5,526 bytes (5.40 KB, 0.01 MB)

3) 결과 비교 및 Lazy Evaluation(지연 평가) 설명

   - 처리 시간(리스트): 5.508550 sec (5508.550 ms) / 더 빠름 (약 1.05x)
   - 처리 시간(제너레이터): 5.779389 sec (5779.389 ms) / 더 느림 (약 1.05x)
   - 처리 시간 차이(리스트 - 제너레이터): -0.270839 sec (-270.839 ms)

   - Peak 메모리(리스트): 409,110,488 bytes (399521.96 KB, 390.16 MB)
   - Peak 메모리(제너레이터): 5,526 bytes (5.40 KB, 0.01 MB)
   - Peak 메모리 차이(리스트 - 제너레이터): 409,104,962 bytes (399516.56 KB, 390.15 MB)

【 Lazy Evaluation(지